# Setup

In [11]:
# Import necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Import Raw Data

## Reference
https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [4]:
# Import General Metrics data, and take a peek
dfRaw_generalMetrics = pd.read_excel('data/raw_data.xlsx', sheet_name='General Metrics');
dfRaw_generalMetrics

,Overall Rating,Total Number of Reviews,Webscraping Datetime
0,3.9,329,2023-01-29 20:55:10


In [5]:
# Import Stars Distribution data, and take a peek
dfRaw_starsDistribution = pd.read_excel('data/raw_data.xlsx', sheet_name='Stars Distribution');
dfRaw_starsDistribution


,Stars Given,Number of Reviews
0,5,182
1,4,53
2,3,21
3,2,15
4,1,58


In [7]:
# Import Reviews data, and take a peek
dfRaw_reviews = pd.read_excel('data/raw_data.xlsx', sheet_name='Reviews');
print(dfRaw_reviews.info());
dfRaw_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 327 non-null    object
 1   Total Reviews Given  327 non-null    object
 2   Time of Review       327 non-null    object
 3   Review               230 non-null    object
 4   Stars Given          322 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB
None


,Name,Total Reviews Given,Time of Review,Review,Stars Given
0,Vielka Duran,Local Guide · 41 reviews,6 hours ago,NaN,5 stars
1,LASHLOVE33,3 reviews,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars
2,Linda Wilke,4 reviews,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars
3,Alex Reyes,2 reviews,a month ago,Response from the owner a month ago,3 stars
4,Hugo Nakamura,Local Guide · 25 reviews,a month ago,"EDIT: The second time coming to the store, the...",5 stars


# Clean & Transform the Raw Reviews Data

## Reference
https://www.learndatasci.com/solutions/python-string-contains/#:~:text=The%20easiest%20and%20most%20effective,can't%20find%20the%20substring.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

https://www.digitalocean.com/community/tutorials/pandas-dataframe-apply-examples

https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

https://www.digitalocean.com/community/tutorials/python-remove-character-from-string

https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/

In [98]:
# Create a copy of raw Reviews data for data cleaning
dfCleaned_reviews = dfRaw_reviews.copy();

# Change name of 'Name' column to 'Reviewer Name'
dfCleaned_reviews = dfCleaned_reviews.rename(columns={'Name':'Reviewer Name'});
dfCleaned_reviews.columns

Index(['Reviewer Name', 'Total Reviews Given', 'Time of Review', 'Review',
       'Stars Given'],
      dtype='object')

In [99]:
# Retrieve reviewer title from 'Total Reviews Given' column 
# and create a new 'Reviewer Title' column
def get_reviewerTitle(df):
    df['Reviewer Title'] = None;
    for i in df.index:
        if '·' in df.loc[i, 'Total Reviews Given']:
            colValue = df.loc[i, 'Total Reviews Given'].split('·');
            df.loc[i, 'Reviewer Title'] = colValue[0].strip();
            df.loc[i, 'Total Reviews Given'] = colValue[1].strip();
    print(df.info());
    return df;

dfCleaned_reviews = get_reviewerTitle(dfCleaned_reviews);
dfCleaned_reviews.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Reviewer Name        327 non-null    object
 1   Total Reviews Given  327 non-null    object
 2   Time of Review       327 non-null    object
 3   Review               230 non-null    object
 4   Stars Given          322 non-null    object
 5   Reviewer Title       203 non-null    object
dtypes: object(6)
memory usage: 15.5+ KB
None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41 reviews,6 hours ago,NaN,5 stars,Local Guide
1,LASHLOVE33,3 reviews,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars,None
2,Linda Wilke,4 reviews,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars,None
3,Alex Reyes,2 reviews,a month ago,Response from the owner a month ago,3 stars,None
4,Hugo Nakamura,25 reviews,a month ago,"EDIT: The second time coming to the store, the...",5 stars,Local Guide


In [100]:
# Clean 'Total Reviews Given' column
dfCleaned_reviews['Total Reviews Given'] = dfCleaned_reviews['Total Reviews Given'].apply(lambda x: x.split(" ")[0]);
dfCleaned_reviews.head()


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41,6 hours ago,NaN,5 stars,Local Guide
1,LASHLOVE33,3,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars,None
2,Linda Wilke,4,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars,None
3,Alex Reyes,2,a month ago,Response from the owner a month ago,3 stars,None
4,Hugo Nakamura,25,a month ago,"EDIT: The second time coming to the store, the...",5 stars,Local Guide


In [101]:
# Clean the 'Total Reviews Given' column
def convert_totalReviewsGivenToNumeric(df):
    # Parse value in 'Total Reviews Given' column to numeric
    # and record which rows are unable to parse to numeric
    index_list = [];
    for i in df.index:
        try:
            df.loc[i, 'Total Reviews Given'] = int(df.loc[i, 'Total Reviews Given']);
        except ValueError:
            print(f"Unable to parse to integer the number of reviews given by {df.loc[i, 'Reviewer Name']}");
            index_list.append(i);
    # From the rows where the values in the 'Total Reviews Given' column 
    # could not be parsed to numeric, remove any instance of ","
    # or equate the value to 0
    for i in index_list:
        if ',' in df.loc[i, 'Total Reviews Given']:
            df.loc[i, 'Total Reviews Given'] = df.loc[i, 'Total Reviews Given'].replace(',','');
        else:
            df.loc[i, 'Total Reviews Given'] = 0;
    df['Total Reviews Given'] = pd.to_numeric(df['Total Reviews Given']);
    print(df.info());
    return df;

dfCleaned_reviews = convert_totalReviewsGivenToNumeric(dfCleaned_reviews);
dfCleaned_reviews.head()


Unable to parse to integer the number of reviews given by MiHyun Kang
Unable to parse to integer the number of reviews given by Alex Passini
Unable to parse to integer the number of reviews given by Armando yaya
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Reviewer Name        327 non-null    object
 1   Total Reviews Given  327 non-null    int64 
 2   Time of Review       327 non-null    object
 3   Review               230 non-null    object
 4   Stars Given          322 non-null    object
 5   Reviewer Title       203 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.5+ KB
None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
0,Vielka Duran,41,6 hours ago,NaN,5 stars,Local Guide
1,LASHLOVE33,3,2 weeks ago,Hit and miss. Staff are either extremely rude ...,2 stars,None
2,Linda Wilke,4,4 weeks ago,Tried calling this boutique to ask some questi...,2 stars,None
3,Alex Reyes,2,a month ago,Response from the owner a month ago,3 stars,None
4,Hugo Nakamura,25,a month ago,"EDIT: The second time coming to the store, the...",5 stars,Local Guide


In [102]:
# Check the rows where the 'Total Reviews Given' column value could not 
# initially be parsed to numeric had appropriate adjustments so that they later were
# parsed correctly

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'MiHyun Kang'])

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'Alex Passini'])

display(dfCleaned_reviews[dfCleaned_reviews['Reviewer Name'] == 'Armando yaya'])

,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
233,MiHyun Kang,0,3 years ago,NaN,4 stars,None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
296,Alex Passini,2441,4 years ago,Very friendly staff and a great selection of c...,5 stars,Local Guide


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
314,Armando yaya,1090,4 years ago,love the coffee,5 stars,Local Guide


In [103]:
# Check the unique values in 'Time of Review' column
dfCleaned_reviews['Time of Review'].unique()

array(['6 hours ago', '2 weeks ago', '4 weeks ago', 'a month ago',
       '2 months ago', '3 months ago', '4 months ago', '5 months ago',
       '6 months ago', '7 months ago', '8 months ago', '9 months ago',
       '11 months ago', 'a year ago', '2 years ago', ' 5 stars ',
       '3 years ago', '4 years ago', ' 1 star '], dtype=object)

In [107]:
# In the above code output, it is evident that there has been some web scraping error
# for the following rows, which require some adjustment such that
# the value for 'Total Reviews Given' is moved to 'Time of Review' column
# and the value for 'Time of Review' is moved to 'Stars Given'
display(dfCleaned_reviews[dfCleaned_reviews['Time of Review'] == ' 5 stars '])

display(dfCleaned_reviews[dfCleaned_reviews['Time of Review'] == ' 1 star '])

,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
100,Jason Wu,2,5 stars,NaN,NaN,None
108,Randy Rhoads,2,5 stars,NaN,NaN,None
171,박유경,3,5 stars,NaN,NaN,None
192,kenny Chow,3,5 stars,NaN,NaN,None


,Reviewer Name,Total Reviews Given,Time of Review,Review,Stars Given,Reviewer Title
323,Jina Lee,4,1 star,NaN,NaN,None
